In [1]:
import backoff
from typing import Any

BACKOFF_CONFIG: dict[str, Any] = {'wait_gen': backoff.expo, 'exception': Exception, 'max_value': 8}

class ClickhouseSettings:
    NODES: str = 'clickhouse-node01:9000,clickhouse-node02:9000,clickhouse-node03:9000,clickhouse-node04:9000'
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode1:
    HOST: str = 'clickhouse-node01'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode2:
    HOST: str = 'clickhouse-node02'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode3:
    HOST: str = 'clickhouse-node03'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

class ClickhouseNode4:
    HOST: str = 'clickhouse-node04'
    PORT: int = 9000
    USER: str = 'user'
    PASSWORD: str = 'password'

CLICKHOUSE_CONFIG = ClickhouseSettings()
NODES = [ClickhouseNode1(), ClickhouseNode2(), ClickhouseNode3(), ClickhouseNode4()]


In [2]:
clickhouse_url = f'jdbc:clickhouse://{NODES[0].HOST}:{NODES[0].PORT}'
properties = {
    'user': NODES[0].USER,
    'password': NODES[0].PASSWORD,
    'driver': 'com.github.housepower.jdbc.ClickHouseDriver'
}

In [3]:
from pyspark.sql import SparkSession

appName='clickhouse_connector'
spark = SparkSession \
    .builder \
    .appName(appName) \
    .master('spark://spark-master:7077') \
    .config('spark.jars', '/opt/clickhouse-native-jdbc-shaded-2.7.1.jar') \
    .getOrCreate()

23/12/07 12:54:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
query = 'SELECT * from default.candles ORDER BY begin DESC'
d = spark.read.format("jdbc") \
    .option('driver', properties.get('driver')) \
    .option('url', clickhouse_url) \
    .option('user', properties.get('user')) \
    .option('password',  properties.get('password')) \
    .option('query', query) \
    .load()

In [6]:
d.take(15)

[Row(id='216b49a7-eb2f-4bb8-885a-ea0b02d00276', secid='ABIO', open=37.07, close=37.07, high=37.07, low=37.07, value=3707.0, volume=100.0, begin=datetime.datetime(2021, 6, 25, 14, 53), end=datetime.datetime(2021, 6, 25, 14, 53, 59), created_at=datetime.datetime(2023, 12, 7, 9, 52, 52)),
 Row(id='dbca6691-e929-42ca-aa1c-4f5845f664e0', secid='ABIO', open=37.01, close=37.0, high=37.01, low=37.0, value=118404.0, volume=3200.0, begin=datetime.datetime(2021, 6, 25, 14, 45), end=datetime.datetime(2021, 6, 25, 14, 45, 59), created_at=datetime.datetime(2023, 12, 7, 9, 52, 52)),
 Row(id='e140d370-fe23-46cf-b808-58c8e169cd42', secid='ABIO', open=37.01, close=37.0, high=37.08, low=37.0, value=25911.0, volume=700.0, begin=datetime.datetime(2021, 6, 25, 14, 42), end=datetime.datetime(2021, 6, 25, 14, 42, 59), created_at=datetime.datetime(2023, 12, 7, 9, 52, 52)),
 Row(id='a0c2bb33-450e-45f4-a201-01c0c914f8ab', secid='ABIO', open=37.0, close=37.02, high=37.02, low=37.0, value=29612.0, volume=800.0, be